# כיוונון עדין של מודלים של Open AI

מחברת זו מבוססת על ההנחיות העדכניות המסופקות בתיעוד [Fine Tuning](https://platform.openai.com/docs/guides/fine-tuning?WT.mc_id=academic-105485-koreyst) של Open AI.

כיוונון עדין משפר את הביצועים של מודלים בסיסיים עבור היישום שלך על ידי אימון חוזר עם נתונים נוספים והקשר הרלוונטי למקרה או לתרחיש הספציפי הזה. שים לב שטכניקות הנדסת פרומפט כמו _למידה עם דוגמאות מועטות_ ו-_יצירה משופרת באמצעות אחזור_ מאפשרות לך לשפר את הפרומפט ברירת המחדל עם נתונים רלוונטיים לשיפור האיכות. עם זאת, גישות אלו מוגבלות על ידי גודל חלון הטוקנים המקסימלי של המודל הבסיסי המיועד.

עם כיוונון עדין, אנו למעשה מאמנים מחדש את המודל עצמו עם הנתונים הנדרשים (מה שמאפשר לנו להשתמש בהרבה יותר דוגמאות ממה שיכול להתאים בחלון הטוקנים המקסימלי) - ומפרסמים גרסה _מותאמת אישית_ של המודל שלא זקוקה עוד לספק דוגמאות בזמן ההסקה. זה לא רק משפר את היעילות של עיצוב הפרומפט שלנו (יש לנו יותר גמישות בשימוש בחלון הטוקנים לדברים אחרים) אלא גם עשוי לשפר את העלויות שלנו (על ידי הפחתת מספר הטוקנים שעלינו לשלוח למודל בזמן ההסקה).

לכיוונון עדין יש 4 שלבים:
1. הכנת נתוני האימון והעלאתם.
1. הרצת משימת האימון לקבלת מודל מכוונן עדין.
1. הערכת המודל המכוין עדין וחזרה לשיפור האיכות.
1. פריסת המודל המכוין עדין להסקה כאשר מרוצים.

שים לב שלא כל המודלים הבסיסיים תומכים בכיוונון עדין - [בדוק את תיעוד OpenAI](https://platform.openai.com/docs/guides/fine-tuning/what-models-can-be-fine-tuned?WT.mc_id=academic-105485-koreyst) למידע העדכני ביותר. ניתן גם לכוונן עדין מודל שכבר כוונן עדין בעבר. במדריך זה, נשתמש ב-`gpt-35-turbo` כמודל הבסיסי המיועד לכיוונון עדין. 

---


### שלב 1.1: הכנת מערך הנתונים שלך

בואו נבנה צ'אטבוט שיעזור לך להבין את הטבלה המחזורית של היסודות על ידי מענה על שאלות לגבי יסוד בלימריק. במדריך הפשוט _הזה_ ניצור רק מערך נתונים לאימון המודל עם כמה דוגמאות תגובה המדגימות את הפורמט הצפוי של הנתונים. במקרה שימוש אמיתי, תצטרך ליצור מערך נתונים עם הרבה יותר דוגמאות. ייתכן גם שתוכל להשתמש במערך נתונים פתוח (לתחום היישום שלך) אם קיים כזה, ולעצב אותו מחדש לשימוש בכוונון עדין.

מכיוון שאנו מתמקדים ב-`gpt-35-turbo` ומחפשים תגובה בסיבוב יחיד (השלמת שיחה), נוכל ליצור דוגמאות באמצעות [הפורמט המוצע הזה](https://platform.openai.com/docs/guides/fine-tuning/preparing-your-dataset?WT.mc_id=academic-105485-koreyst) המשקף את דרישות השלמת השיחה של OpenAI. אם אתה מצפה לתוכן שיחה רב-סיבובי, תשתמש ב-[פורמט הדוגמאות הרב-סיבובי](https://platform.openai.com/docs/guides/fine-tuning/multi-turn-chat-examples?WT.mc_id=academic-105485-koreyst) הכולל פרמטר `weight` כדי לסמן אילו הודעות יש להשתמש בהן (או לא) בתהליך הכוונון העדין.

נשתמש כאן בפורמט הפשוט של סיבוב יחיד למדריך שלנו. הנתונים הם בפורמט [jsonl](https://jsonlines.org/?WT.mc_id=academic-105485-koreyst) עם רשומה אחת בכל שורה, שכל אחת מיוצגת כאובייקט בפורמט JSON. הקטע למטה מציג 2 רשומות כדוגמה - ראה את [training-data.jsonl](../../../../../18-fine-tuning/python/openai/training-data.jsonl) למערך דוגמאות מלא (10 דוגמאות) שנשתמש בו במדריך הכוונון העדין שלנו. **הערה:** כל רשומה _חייבת_ להיות מוגדרת בשורה אחת (ולא מפוצלת על פני שורות כפי שנהוג בקובץ JSON מעוצב).

```json
{ "messages": [{"role": "system", "content": "Elle is a factual chatbot that answers questions about elements in the periodic table with a limerick"}, {"role": "user", "content": "Tell me about Gallium"}, {"role": "assistant", "content": "Gallium, oh gallium, so light - Melts in your hand, oh what a sight - At 86 degrees - Its liquid with ease - And in semiconductors, it's out of sight"}]}
{ "messages": [{"role": "system", "content": "Elle is a factual chatbot that answers questions about elements in the periodic table with a limerick"}, {"role": "user", "content": "Tell me about Hydrogen"}, {"role": "assistant", "content": "Hydrogen, the first in the line - The lightest of all, so divine - It's in water, you see - And in stars, it's the key - The universe's most common sign"}]}
```
  
בשימוש אמיתי תזדקק למערך דוגמאות גדול בהרבה לתוצאות טובות - הפשרה תהיה בין איכות התגובות לבין הזמן/העלויות של הכוונון העדין. אנו משתמשים במערך קטן כדי שנוכל להשלים את הכוונון במהירות ולהמחיש את התהליך. ראה [דוגמת OpenAI Cookbook הזו](https://github.com/openai/openai-cookbook/blob/main/examples/How_to_finetune_chat_models.ipynb?WT.mc_id=academic-105485-koreyst) למדריך כוונון עדין מורכב יותר.


---

### שלב 1.2 העלאת מערך הנתונים שלך

העלה את הנתונים באמצעות ממשק ה-API של הקבצים [כמתואר כאן](https://platform.openai.com/docs/guides/fine-tuning/upload-a-training-file). שים לב שכדי להריץ את הקוד הזה, עליך לבצע את השלבים הבאים קודם:
 - התקנת חבילת הפייתון `openai` (ודא שאתה משתמש בגרסה >=0.28.0 עבור התכונות העדכניות ביותר)
 - הגדרת משתנה הסביבה `OPENAI_API_KEY` למפתח ה-API של OpenAI שלך
למידע נוסף, ראה את [מדריך ההתקנה](./../../../00-course-setup/02-setup-local.md?WT.mc_id=academic-105485-koreyst) שסופק לקורס.

כעת, הרץ את הקוד כדי ליצור קובץ להעלאה מקובץ ה-JSONL המקומי שלך.


In [24]:
from openai import OpenAI
client = OpenAI()

ft_file = client.files.create(
  file=open("./training-data.jsonl", "rb"),
  purpose="fine-tune"
)

print(ft_file)
print("Training File ID: " + ft_file.id)

FileObject(id='file-JdAJcagdOTG6ACNlFWzuzmyV', bytes=4021, created_at=1715566183, filename='training-data.jsonl', object='file', purpose='fine-tune', status='processed', status_details=None)
Training File ID: file-JdAJcagdOTG6ACNlFWzuzmyV


---

### שלב 2.1: צור את משימת הכוונון המדויק עם ה-SDK


In [25]:
from openai import OpenAI
client = OpenAI()

ft_filejob = client.fine_tuning.jobs.create(
  training_file=ft_file.id, 
  model="gpt-3.5-turbo"
)

print(ft_filejob)
print("Fine-tuning Job ID: " + ft_filejob.id)

FineTuningJob(id='ftjob-Usfb9RjasncaZ5Cjbuh1XSCh', created_at=1715566184, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs='auto', batch_size='auto', learning_rate_multiplier='auto'), model='gpt-3.5-turbo-0125', object='fine_tuning.job', organization_id='org-EZ6ag0n0S6Zm8eV9BSWKmE6l', result_files=[], seed=830529052, status='validating_files', trained_tokens=None, training_file='file-JdAJcagdOTG6ACNlFWzuzmyV', validation_file=None, estimated_finish=None, integrations=[], user_provided_suffix=None)
Fine-tuning Job ID: ftjob-Usfb9RjasncaZ5Cjbuh1XSCh


---

### שלב 2.2: בדוק את מצב העבודה

הנה כמה דברים שאתה יכול לעשות עם ה-API של `client.fine_tuning.jobs`:
- `client.fine_tuning.jobs.list(limit=<n>)` - רשום את n עבודות הכוונון האחרונות
- `client.fine_tuning.jobs.retrieve(<job_id>)` - קבל פרטים של עבודת כוונון ספציפית
- `client.fine_tuning.jobs.cancel(<job_id>)` - בטל עבודת כוונון
- `client.fine_tuning.jobs.list_events(fine_tuning_job_id=<job_id>, limit=<b>)` - רשום עד n אירועים מהעבודה
- `client.fine_tuning.jobs.create(model="gpt-35-turbo", training_file="your-training-file.jsonl", ...)`

השלב הראשון בתהליך הוא _אימות קובץ האימון_ כדי לוודא שהנתונים בפורמט הנכון.


In [26]:
from openai import OpenAI
client = OpenAI()

# List 10 fine-tuning jobs
client.fine_tuning.jobs.list(limit=10)

# Retrieve the state of a fine-tune
client.fine_tuning.jobs.retrieve(ft_filejob.id)

# List up to 10 events from a fine-tuning job
client.fine_tuning.jobs.list_events(fine_tuning_job_id=ft_filejob.id, limit=10)

SyncCursorPage[FineTuningJobEvent](data=[FineTuningJobEvent(id='ftevent-GkWiDgZmOsuv4q5cSTEGscY6', created_at=1715566184, level='info', message='Validating training file: file-JdAJcagdOTG6ACNlFWzuzmyV', object='fine_tuning.job.event', data={}, type='message'), FineTuningJobEvent(id='ftevent-3899xdVTO3LN7Q7LkKLMJUnb', created_at=1715566184, level='info', message='Created fine-tuning job: ftjob-Usfb9RjasncaZ5Cjbuh1XSCh', object='fine_tuning.job.event', data={}, type='message')], object='list', has_more=False)

In [30]:
# Once the training data is validated
# Track the job status to see if it is running and when it is complete
from openai import OpenAI
client = OpenAI()

response = client.fine_tuning.jobs.retrieve(ft_filejob.id)

print("Job ID:", response.id)
print("Status:", response.status)
print("Trained Tokens:", response.trained_tokens)

Job ID: ftjob-Usfb9RjasncaZ5Cjbuh1XSCh
Status: running
Trained Tokens: None


---

### שלב 2.3: עקוב אחר אירועים כדי לנטר התקדמות


In [44]:
# You can also track progress in a more granular way by checking for events
# Refresh this code till you get the `The job has successfully completed` message
response = client.fine_tuning.jobs.list_events(ft_filejob.id)

events = response.data
events.reverse()

for event in events:
    print(event.message)

Step 85/100: training loss=0.14
Step 86/100: training loss=0.00
Step 87/100: training loss=0.00
Step 88/100: training loss=0.07
Step 89/100: training loss=0.00
Step 90/100: training loss=0.00
Step 91/100: training loss=0.00
Step 92/100: training loss=0.00
Step 93/100: training loss=0.00
Step 94/100: training loss=0.00
Step 95/100: training loss=0.08
Step 96/100: training loss=0.05
Step 97/100: training loss=0.00
Step 98/100: training loss=0.00
Step 99/100: training loss=0.00
Step 100/100: training loss=0.00
Checkpoint created at step 80 with Snapshot ID: ft:gpt-3.5-turbo-0125:bitnbot::9OFWyyF2:ckpt-step-80
Checkpoint created at step 90 with Snapshot ID: ft:gpt-3.5-turbo-0125:bitnbot::9OFWyzhK:ckpt-step-90
New fine-tuned model created: ft:gpt-3.5-turbo-0125:bitnbot::9OFWzNjz
The job has successfully completed


### שלב 2.4: הצגת הסטטוס בלוח הבקרה של OpenAI


אתה יכול גם לצפות במצב על ידי ביקור באתר OpenAI וחקר של מדור _Fine-tuning_ בפלטפורמה. זה יראה לך את מצב העבודה הנוכחית, ויאפשר לך גם לעקוב אחרי היסטוריית הריצות של ביצועי עבודות קודמות. בתמונה זו, ניתן לראות שהריצה הקודמת נכשלה, והריצה השנייה הצליחה. להקשר, זה קרה כאשר הריצה הראשונה השתמשה בקובץ JSON עם רשומות בפורמט שגוי - לאחר התיקון, הריצה השנייה הושלמה בהצלחה והפכה את המודל לזמין לשימוש.

![Fine-tuning job status](../../../../../translated_images/he/fine-tuned-model-status.563271727bf7bfba.png)


אתה יכול גם לצפות בהודעות הסטטוס ובמדדים על ידי גלילה למטה עוד בלוח המחוונים הוויזואלי כפי שמוצג:

| הודעות | מדדים |
|:---|:---|
| ![Messages](../../../../../translated_images/he/fine-tuned-messages-panel.4ed0c2da5ea1313b.png) |  ![Metrics](../../../../../translated_images/he/fine-tuned-metrics-panel.700d7e4995a65229.png)|


---

### שלב 3.1: שליפת מזהה ובדיקת מודל מכוונן בקוד


In [46]:
# Retrieve the identity of the fine-tuned model once ready
response = client.fine_tuning.jobs.retrieve(ft_filejob.id)
fine_tuned_model_id = response.fine_tuned_model
print("Fine-tuned Model ID:", fine_tuned_model_id)

Fine-tuned Model ID: ft:gpt-3.5-turbo-0125:bitnbot::9OFWzNjz


In [47]:
# You can then use that model to generate completions from the SDK as shown
# Or you can load that model into the OpenAI Playground (in the UI) to validate it from there.
from openai import OpenAI
client = OpenAI()

completion = client.chat.completions.create(
  model=fine_tuned_model_id,
  messages=[
    {"role": "system", "content": "You are Elle, a factual chatbot that answers questions about elements in the periodic table with a limerick"},
    {"role": "user", "content": "Tell me about Strontium"},
  ]
)
print(completion.choices[0].message)

ChatCompletionMessage(content="Strontium, a metal so bright - It's in fireworks, a dazzling sight - It's in bones, you see - And in tea, it's the key - It's the fortieth, so pure, that's the right", role='assistant', function_call=None, tool_calls=None)


---

### שלב 3.2: טעינת ובדיקת מודל מותאם אישית ב-Playground

כעת ניתן לבדוק את המודל המותאם בשתי דרכים. ראשית, ניתן לבקר ב-Playground ולהשתמש בתפריט הנפתח Models כדי לבחור את המודל המותאם החדש מתוך האפשרויות המופיעות. האפשרות השנייה היא להשתמש באפשרות "Playground" המוצגת בלוח Fine-tuning (ראה צילום המסך למעלה) אשר מפעילה את תצוגת ה_השוואה_ הבאה המציגה את גרסאות המודל הבסיסי והמודל המותאם זה לצד זה להערכה מהירה.

![Fine-tuning job status](../../../../../translated_images/he/fine-tuned-playground-compare.56e06f0ad8922016.png)

פשוט מלא את הקשר המערכת ששימש בנתוני האימון שלך וספק את שאלת הבדיקה שלך. תבחין ששני הצדדים מתעדכנים עם אותו הקשר ושאלה זהים. הפעל את ההשוואה ותראה את ההבדל בתגובות ביניהם. _שים לב כיצד המודל המותאם מציג את התגובה בפורמט שסיפקת בדוגמאות שלך בעוד שהמודל הבסיסי פשוט עוקב אחרי הפקודה של המערכת_.

![Fine-tuning job status](../../../../../translated_images/he/fine-tuned-playground-launch.5a26495c983c6350.png)

תבחין שגם ההשוואה מספקת את ספירת הטוקנים עבור כל מודל, ואת הזמן שנדרש להסקת המסקנות. **דוגמה ספציפית זו היא פשוטה ומטרתה להראות את התהליך אך אינה משקפת מערך נתונים או תרחיש אמיתי**. ייתכן שתבחין ששני הדוגמאות מציגות את אותו מספר טוקנים (הקשר המערכת וההנחיה זהים) כאשר המודל המותאם לוקח יותר זמן להסקת מסקנות (מודל מותאם אישית).

בתרחישים אמיתיים, לא תשתמש בדוגמה פשוטה כזו, אלא תתאים את המודל לנתונים אמיתיים (למשל, קטלוג מוצרים לשירות לקוחות) שבהם איכות התגובה תהיה ברורה הרבה יותר. בהקשר _זה_, קבלת איכות תגובה שווה ערך עם המודל הבסיסי תדרוש הנדסת פקודות מותאמת אישית שתגדיל את השימוש בטוקנים ואולי גם את זמן העיבוד להסקת מסקנות. _כדי לנסות זאת, עיין בדוגמאות ההתאמה האישית בספר הבישול של OpenAI כדי להתחיל._

---


---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**כתב ויתור**:  
מסמך זה תורגם באמצעות שירות תרגום מבוסס בינה מלאכותית [Co-op Translator](https://github.com/Azure/co-op-translator). למרות שאנו שואפים לדיוק, יש לקחת בחשבון כי תרגומים אוטומטיים עלולים להכיל שגיאות או אי-דיוקים. המסמך המקורי בשפת המקור שלו הוא המקור הסמכותי. למידע קריטי מומלץ להשתמש בתרגום מקצועי על ידי אדם. אנו לא נושאים באחריות לכל אי-הבנה או פרשנות שגויה הנובעת משימוש בתרגום זה.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
